# Trabajo Tesis: Preprocesamiento

### Librerias y bases

In [ ]:
import numpy as np
import pandas as pd
import os
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm


In [ ]:
os.chdir('c:/Users/artur/OneDrive/Documents/TrabajoTesis') 

In [148]:
df = pd.read_excel('Cluster/Faenas25.xlsx')

In [149]:
# Limpiar nombres de columnas
df.columns = ['RutEmpresa' if col == 'RutEmpresa' else col.lstrip() for col in df.columns]

# Limpiar espacios dentro de las columnas (excepto RutEmpresa)
for col in df.select_dtypes(include='object').columns:
    if col != 'RutEmpresa':
        df[col] = df[col].str.strip()


In [150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28319 entries, 0 to 28318
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RutEmpresa                  28319 non-null  object 
 1   NombreEmpresa               28319 non-null  object 
 2   RegionFaena                 28319 non-null  object 
 3   ProvinciaFaena              28319 non-null  object 
 4   ComunaFaena                 28319 non-null  object 
 5   NombreFaena                 28318 non-null  object 
 6   CategoriaFaena              28319 non-null  object 
 7   IdFaena                     28319 non-null  int64  
 8   RegionInstalacion           28319 non-null  object 
 9   ProvinciaInstalacion        28319 non-null  object 
 10  ComunaInstalacion           28319 non-null  object 
 11  NombreInstalacion           28319 non-null  object 
 12  IdTipoInstalacion           28319 non-null  int64  
 13  TipoInstalacion             283

### EDA preliminar

In [151]:
print(df['TipoInstalacion'].unique())


['TALLERES Y MAESTRANZA' 'EXPLORACION DE SUPERFICIE' 'MINA SUBTERRANEA'
 'MINA RAJO ABIERTO' 'DEPÓSITO RMM - ESTÉRILES' 'BODEGA' 'CAMPAMENTO'
 'POLVORIN' 'ADMINISTRACIÓN - OFICINA' 'PLANTA CHANCADO'
 'DEPÓSITO RMM - MINERAL DE BAJA LEY' 'LIXIVIACION' 'PLANTA CONCENTRADORA'
 'SUB-ESTACION ELECTRICA' 'DEPÓSITO - TRANQUE DE RELAVES'
 'CANCHA DE ACOPIO' 'MUESTRERA Y/O LABORATORIO' 'PUERTO DE EMBARQUE'
 'PLANTA DIMENSIONAMIENTO Y CORTE' 'PLANTA MOLIENDA' 'PLANTA ESPESADORES'
 'PLANTA FILTRO' 'PILAS DE FLOTACION' 'DEPÓSITO - FILTRADOS'
 'PLANTA EXTRACCIÓN POR SOLVENTES' 'PISCINAS DE EMERGENCIA'
 'ESTACION DE COMPRESORES' 'ESPESADORES' 'MINERODUCTO'
 'ESTACION DE BOMBEO' 'FUNDICION' 'CASA DE FUERZA' 'SALA DE CONTROL'
 'PLANTA DE CONVERSION' 'PLANTA DE ACIDO' 'CORREA TRANSPORTADORA'
 'CENTRAL - GAS' 'BATERIA DE RECEPCIÓN - PETRÓLEO' 'CALENTADOR - PETRÓLEO'
 'POZO DE PETROLEO' 'DEPÓSITO - EMBALSES'
 'TRAMPAS DE RECEPCIÓN/LANZAMIENTO HC' 'POZO DE GAS'
 'PLANTA QUÍMICA DE CONVERSIÓN' 'POZOS DE EX

In [152]:
from IPython.display import display, HTML

html = "<div style='height:300px; overflow:auto;'>" + df['TipoInstalacion'].value_counts().to_frame().to_html() + "</div>"
display(HTML(html))

,count
TipoInstalacion,
MINA SUBTERRANEA,9100
MINA RAJO ABIERTO,3716
DEPÓSITO RMM - ESTÉRILES,2884
CAMPAMENTO,1766
EXPLORACION DE SUPERFICIE,1659
POLVORIN,1595
DEPÓSITO RMM - MINERAL DE BAJA LEY,736
PLANTA CONCENTRADORA,672
TALLERES Y MAESTRANZA,644


In [153]:
df['Estado'].value_counts()

Estado
ACTIVA                                 10834
ABANDONADA                             10134
IRREGULAR OPERATIVA                     2204
INACTIVA                                1508
EN REVISIÓN DE PROYECTO                 1358
EXPLORACIÓN FINALIZADA                   715
CIERRE FINAL                             577
PROYECTADA                               375
CIERRE TEMPORAL                          331
PARALIZADA POR SANCION                   242
PARALIZACIÓN POR MEDIDA PROVISIONAL       41
Name: count, dtype: int64

In [154]:
df['CategoriaFaena'].value_counts()

CategoriaFaena
CATEGORIA D      20100
CATEGORIA C       3601
CATEGORIA A       2941
CATEGORIA B        849
SIN CATEGORIA      828
Name: count, dtype: int64

In [155]:
df['RecursoPrimarioInstalacion']

0           COBRE
1           COBRE
2           COBRE
3          HIERRO
4           COBRE
           ...   
28314       COBRE
28315    SILICEOS
28316    SILICEOS
28317       COBRE
28318         ORO
Name: RecursoPrimarioInstalacion, Length: 28319, dtype: object

### Limpieza

In [156]:
df = df[df['RecursoPrimarioInstalacion'] == 'COBRE']
df.shape

(21158, 27)

In [157]:
df

,RutEmpresa,NombreEmpresa,RegionFaena,ProvinciaFaena,ComunaFaena,NombreFaena,CategoriaFaena,IdFaena,RegionInstalacion,ProvinciaInstalacion,...,IdInstalacion,Norte,Este,Cota,Huso,Datum,IdEstado,Estado,Latitud,Longitud
0,61703000-4,EMPRESA NACIONAL DE MINERIA (ENAMI),III,HUASCO,VALLENAR,PLANTA DE BENEFICIO DE MINERALES - VALLENAR,CATEGORIA B,153576,ATACAMA,HUASCO,...,2000016078,6839116.0,330003.0,538,19,PSAD-56,8,ACTIVA,-28.566725,-70.740064
1,76125921-0,CUPRUM RESOURCES CHILE SPA,IV,ELQUI,LA HIGUERA,LA VERDE,CATEGORIA D,20012162,COQUIMBO,ELQUI,...,2000016187,6743203.0,336885.0,1200,19,PSAD-56,169,INACTIVA,-29.432920,-70.683699
2,76362163-4,MINERA RUPERTO PATRICIO ORTIZ PALOMINOS,VI,COLCHAGUA,LOLOL,MINA LA VENUS,CATEGORIA D,20013211,LIBERTADOR GENERAL BERNARDO OHIGGINS,COLCHAGUA,...,2000017211,6160230.0,271326.0,134,19,PSAD-56,12,ABANDONADA,-34.677189,-71.498091
4,6891906-1,GARY NELSON MERCADO MALBRAN,III,CHAÑARAL,CHAÑARAL,DIEGO DAKAR 1/10,CATEGORIA D,20013237,ATACAMA,CHAÑARAL,...,2000018057,7078335.0,359170.0,531,19,PSAD-56,8,ACTIVA,-26.411292,-70.414104
12,76448510-6,SLM CASUALIDAD PRIMERA DE EL MANZANO,IV,ELQUI,ANDACOLLO,CASUALIDAD UNO,CATEGORIA D,20015093,COQUIMBO,ELQUI,...,2000019076,6657100.0,299764.0,917,19,PSAD-56,2169,IRREGULAR OPERATIVA,-30.204069,-71.082232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28311,78131832-9,BALFENI SOLUTIONS SPA,IV,CHOAPA,CANELA,FAENA DON MARIO,CATEGORIA D,20389551,COQUIMBO,CHOAPA,...,2000557972,6544556.0,284169.0,1277,19,PSAD-56,5170,EN REVISIÓN DE PROYECTO,-31.216132,-71.267759
28312,78131832-9,BALFENI SOLUTIONS SPA,IV,CHOAPA,CANELA,FAENA DON MARIO,CATEGORIA D,20389551,COQUIMBO,CHOAPA,...,2000557973,6544044.0,284421.0,1199,19,PSAD-56,5170,EN REVISIÓN DE PROYECTO,-31.220795,-71.265226
28313,6188182-4,EDILIO GOMEZ OYARZUN,IV,CHOAPA,CANELA,LA ROSARIO 1 AL 20,CATEGORIA D,20346577,COQUIMBO,CHOAPA,...,2000551067,6507500.0,277700.0,458,19,PSAD-56,5170,EN REVISIÓN DE PROYECTO,-31.548972,-71.343902
28314,78110627-5,C & C MINERIA SpA,II,EL LOA,CALAMA,LA DESPRECIADA IV 1 AL 1O,CATEGORIA D,20388642,ANTOFAGASTA,EL LOA,...,2000554937,7451789.0,545590.0,3250,19,PSAD-56,5170,EN REVISIÓN DE PROYECTO,-23.044930,-68.557016


In [158]:
df = df.drop(columns=[
    'RegionFaena', 'ProvinciaFaena', 'ComunaFaena', 'NombreFaena', 'IdFaena',
    'RegionInstalacion', 'NombreInstalacion', 'RecursoMineroInstalacion',
    'TipoRecursoInstalacion', 'IdInstalacion', 'Norte', 'Este', 'Datum'
])

### Ciudades

In [163]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm

# Take a 10% random sample

df_sample = df.sample(frac=0.1, random_state=42).copy() 

# Set up Nominatim (OSM)
geolocator = Nominatim(user_agent="city_locator_sample")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)  # respect API limits

# Function to get nearest city

def get_city(lat, lon):
    try:
        location = reverse((lat, lon), exactly_one=True)
        if location and "address" in location.raw:
            address = location.raw["address"]
            return (
                address.get("city")
                or address.get("town")
                or address.get("village")
                or address.get("hamlet")
                or address.get("municipality")
                or address.get("county")
            )
        return None
    except Exception:
        return None


# Apply to sample

tqdm.pandas()
df_sample["Closest_City"] = df_sample.progress_apply(
    lambda row: get_city(row["Latitud"], row["Longitud"]), axis=1
)

# Inspect results
print(df_sample.head())


100%|██████████| 2116/2116 [36:00<00:00,  1.02s/it]

       RutEmpresa                           NombreEmpresa CategoriaFaena  \
14100  76043345-4                           SCM CAMARONES    CATEGORIA D   
23857  10267925-3                      OMAR MUÑOZ ALVAREZ    CATEGORIA D   
26455  76828678-7  MINERA JESSICA PAMELA PEREZ SILVA EIRL    CATEGORIA D   
17161  76102677-1                         MINERA HMC S.A.    CATEGORIA A   
1557   12774379-7           JULIO CESAR CONTRERAS ELGUETA    CATEGORIA D   

      ProvinciaInstalacion ComunaInstalacion  IdTipoInstalacion  \
14100                ARICA             ARICA               8073   
23857               CHOAPA            CANELA               1291   
26455              PETORCA           CABILDO               8076   
17161          ANTOFAGASTA        MEJILLONES               1350   
1557            SAN FELIPE            CATEMU               1306   

                       TipoInstalacion RecursoPrimarioInstalacion  Cota  Huso  \
14100                 CANCHA DE ACOPIO                     

In [165]:
from IPython.display import display, HTML

html = "<div style='height:300px; overflow:auto;'>" + df_sample['Closest_City'].value_counts().to_frame().to_html() + "</div>"
display(HTML(html))

,count
Closest_City,
Copiapó,181
Cabildo,129
Taltal,122
Ovalle,100
Chañaral,97
Tierra Amarilla,92
Vallenar,88
Diego de Almagro,77
Petorca,77


In [166]:
import requests
from tqdm import tqdm
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Get city coordinates (once per unique city)
geolocator = Nominatim(user_agent="truck_routing_demo")
reverse = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def get_city_coords(city_name):
    try:
        location = geolocator.geocode(city_name + ", Chile")  # add country context
        if location:
            return location.latitude, location.longitude
    except:
        return None, None
    return None, None

# Create a mapping of city -> (lat, lon)
cities = df_sample["Closest_City"].dropna().unique()
city_coords = {}
for c in tqdm(cities, desc="Geocoding city centers"):
    lat, lon = get_city_coords(c)
    city_coords[c] = (lat, lon)

# Function to query OSRM
def get_travel_time(lat1, lon1, city):
    lat2, lon2 = city_coords.get(city, (None, None))
    if lat2 is None or lon2 is None:
        return None
    url = f"https://router.project-osrm.org/route/v1/driving/{lon1},{lat1};{lon2},{lat2}?overview=false"
    try:
        response = requests.get(url)
        data = response.json()
        if data.get("routes"):
            duration_sec = data["routes"][0]["duration"]
            return duration_sec / 60  # convert to minutes
    except Exception:
        return None
    return None

# Apply to DataFrame
tqdm.pandas()
df_sample["TravelTime_Truck_min"] = df_sample.progress_apply(
    lambda row: get_travel_time(row["Latitud"], row["Longitud"], row["Closest_City"]),
    axis=1
)

# Done!
print(df_sample[["Latitud", "Longitud", "Closest_City", "TravelTime_Truck_min"]].head())


100%|██████████| 2116/2116 [35:22<00:00,  1.00s/it]

         Latitud   Longitud Closest_City  TravelTime_Truck_min
14100 -18.766115 -70.305866        Arica             64.160000
23857 -31.187356 -71.307579       Canela             84.291667
26455 -32.413654 -70.932892      Cabildo             28.773333
17161 -22.676943 -70.189357   Mejillones             84.641667
1557  -32.682402 -70.976678       Catemu             34.590000


In [167]:
df_sample

,RutEmpresa,NombreEmpresa,CategoriaFaena,ProvinciaInstalacion,ComunaInstalacion,IdTipoInstalacion,TipoInstalacion,RecursoPrimarioInstalacion,Cota,Huso,IdEstado,Estado,Latitud,Longitud,Closest_City,TravelTime_Truck_min
14100,76043345-4,SCM CAMARONES,CATEGORIA D,ARICA,ARICA,8073,CANCHA DE ACOPIO,COBRE,181,19,12,ABANDONADA,-18.766115,-70.305866,Arica,64.160000
23857,10267925-3,OMAR MUÑOZ ALVAREZ,CATEGORIA D,CHOAPA,CANELA,1291,MINA SUBTERRANEA,COBRE,1100,19,8,ACTIVA,-31.187356,-71.307579,Canela,84.291667
26455,76828678-7,MINERA JESSICA PAMELA PEREZ SILVA EIRL,CATEGORIA D,PETORCA,CABILDO,8076,EXPLORACIÓN SUBTERRÁNEA,COBRE,475,19,8,ACTIVA,-32.413654,-70.932892,Cabildo,28.773333
17161,76102677-1,MINERA HMC S.A.,CATEGORIA A,ANTOFAGASTA,MEJILLONES,1350,TALLERES Y MAESTRANZA,COBRE,765,19,8,ACTIVA,-22.676943,-70.189357,Mejillones,84.641667
1557,12774379-7,JULIO CESAR CONTRERAS ELGUETA,CATEGORIA D,SAN FELIPE,CATEMU,1306,PLANTA EXTRACCIÓN POR SOLVENTES,COBRE,789,19,12,ABANDONADA,-32.682402,-70.976678,Catemu,34.590000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4605,76726970-6,SOCIEDAD MINERA LA ESTRELLA NORTE,CATEGORIA D,ELQUI,LA HIGUERA,1291,MINA SUBTERRANEA,COBRE,485,19,12,ABANDONADA,-29.335595,-71.109707,La Higuera,24.631667
23484,11506964-0,SAUL CHIRINO VALENZUELA,CATEGORIA C,CHAÑARAL,DIEGO DE ALMAGRO,1337,DEPÓSITO RMM - ESTÉRILES,COBRE,1905,19,8,ACTIVA,-26.867926,-70.033074,Copiapó,120.755000
17866,6062657-K,VICTOR SAAVEDRA NAVARRETE,SIN CATEGORIA,IQUIQUE,ALTO HOSPICIO,1292,MINA RAJO ABIERTO,COBRE,760,19,8,ACTIVA,-20.176907,-70.097185,Alto Hospicio,32.471667
19928,8382259-7,OPIR TABITA CUEVAS SILVA,CATEGORIA D,HUASCO,FREIRINA,1349,CAMPAMENTO,COBRE,620,19,8,ACTIVA,-28.609166,-70.986205,Freirina,54.183333


In [168]:
df_sample.to_csv('data/df_sample.csv', index=False)